# Тестовое задание на знание SQL и T-SQL

##### Александр Тиунов
##### 2022 г.

<h1>Оглавление<span class="tocSkip"></span></h1>

*  [Задачи](#Задачи)
*  [Поиск решения](#Поиск-решения)
*  [Задание 1](#Задание-1)
    *  [Шаг 1: импортируем библиотеки](#Шаг-1:-импортируем-библиотеки)
    *  [Шаг 2: загрузка и изучение данных](#Шаг-2:-загрузка-и-изучение-данных)
    -  [Шаг 3: проверка данных](#Шаг-3:-проверка-данных)
    *  [Шаг 4: поиск ошибки](#Шаг-4:-поиск-ошибки)
    * [Ответ на Задание 1](#Ответ-на-Задание-1)
*  [Задание 2](#Задание-2)
    * [Ответ на Задание 2](#Ответ-на-Задание-2)
*  [Задание 3](#Задание-3)
    * [Ответ на Задание 3](#Ответ-на-Задание-3)
*  [Задание 4](#Задание-4)
    * [Ответ на Задание 4](#Ответ-на-Задание-4)

# Задачи

1. Заказчик утверждает, что продажи с НДС по сети, по группе товаров 'Биологически активные добавки’, в июне 2017 составили: 1 782 949.10 в рублях и 6761.10 в штуках.
Процедура ETL исполнителя возвращает другие данные.
Необходимо найти и устранить возможные причины расхождений.
Пример запуска процедуры: exec sp_report_1 @date_from = '2017-06-01', @date_to = '2017-06-30', @good_group_name = N'Биологически активные добавки'
			 
2. В результат процедуры необходимо добавить показатели: 
				-Средняя цена закупки руб., без НДС
				-Маржа руб. без НДС
				-Наценка % без НДС				
3. Добавить возможность фильтрации по нескольким группам товаров одновременно. 
Пример входных данных: 'Биологически активные добавки,Косметические средства'
Пример запуска процедуры после модернизации: exec sp_report_1 @date_from = '2017-06-01', @date_to = '2017-06-30', @good_group_name = 'Биологически активные добавки,Косметические средства'
			
4. Вывести долю продаж товара в %, относительно общего объема продаж в рублях в разрезе день/магазин/группа товаров, затем отсортировать выборку по убыванию показателя (нужна доля от продаж позиции в аптеке от продажи группы товара в этой же аптеке). Показатель добавить в текущую процедуру формирования данных для отчета. Расчет не должен выполняться с помощью вложенных запросов и CTE. Необходимо использовать функции Transact-SQL.


# Поиск решения

## Задание 1
Заказчик утверждает, что продажи с НДС по сети, по группе товаров 'Биологически активные добавки’, в июне 2017 составили: 1 782 949.10 в рублях и 6761.10 в штуках. Процедура ETL исполнителя возвращает другие данные. Необходимо найти и устранить возможные причины расхождений. Пример запуска процедуры: exec sp_report_1 @date_from = '2017-06-01', @date_to = '2017-06-30', @good_group_name = N'Биологически активные добавки'

### Шаг 1: импортируем библиотеки

In [1]:
import urllib
from sqlalchemy import create_engine
import sqlalchemy as sqla
import pandas as pd

params = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};SERVER=localhost;DATABASE=DATABASE_NAME;UID=USER_NAME;PWD=Password")
engine = sqla.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Шаг 2: загрузка и изучение данных

В нашем распоряжении следующие таблицы

In [2]:
pd.read_sql('select * FROM INFORMATION_SCHEMA.TABLES', engine)

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
0,TestTaskDB,dbo,dim_cash_register,BASE TABLE
1,TestTaskDB,dbo,fct_cheque,BASE TABLE
2,TestTaskDB,dbo,dim_goods,BASE TABLE
3,TestTaskDB,dbo,dim_stores,BASE TABLE
4,TestTaskDB,dbo,dim_date,BASE TABLE


Посмотрим содержимое таблиц

In [3]:
pd.read_sql('select * FROM dim_goods', engine)

,good_id,good_name,group_id,group_name,parent_group_id,parent_group_name
0,245427,Виши део-ролик д/очень чувствит кожи 50мл,5.0,Косметические средства,NaN,None
1,102018,Фиточай Алтайский кедр клетчатка алтайская 90х...,1.0,Биологически активные добавки,NaN,None
2,1553,Сироп боярышник с черноплодной рябиной на саха...,1.0,Биологически активные добавки,NaN,None
3,113548,Натура сиберика крем д/ног д/ежедневного ухода...,5.0,Косметические средства,NaN,None
4,149074,З/паста детская Новый жемчуг женьшень 50мл,5.0,Косметические средства,NaN,None
...,...,...,...,...,...,...
25825,296650,Очки корригирующие-8899 ручка широк золот (мет...,NaN,None,NaN,None
25826,296816,"Очки корригирующие-2038 М3 голуб. (пласт.жен,,...",NaN,None,NaN,None
25827,297861,Салфетки влажные д/интим гигиены Кэфри хлопок ...,NaN,None,NaN,None
25828,296733,"Очки корригирующие-6008 черная (пласт.унив,лин...",NaN,None,NaN,None


Таблица dim_goods содержит данные о товарах. Проверим, все ли строки уникальны.

In [4]:
pd.read_sql('select * FROM dim_goods', engine).duplicated().sum()

3

В таблице dim_goods есть дубликаты записей, и это является ошибкой.

In [5]:
pd.read_sql('select * FROM fct_cheque order by date_id ', engine)

,date_id,store_id,good_id,cash_register_id,quantity,cost_net,cost_grs,sale_net,sale_grs
0,20170101,25,139652,9,1.0,138.3600,152.1960,201.1640,215.0
1,20170101,37,20467,14,1.0,108.5600,119.4160,159.1440,170.0
2,20170101,3,125510,1,1.0,286.8000,315.4800,456.3200,485.0
3,20170101,43,175520,16,1.0,78.9000,86.7900,122.1100,130.0
4,20170101,25,153416,9,2.0,18.2728,20.1000,58.1728,60.0
...,...,...,...,...,...,...,...,...,...
895322,20171231,29,224355,11,1.0,139.6000,153.5600,182.0400,196.0
895323,20171231,9,297562,4,1.0,113.5500,133.9890,160.0610,180.5
895324,20171231,43,142073,16,1.0,84.3700,92.8070,166.5630,175.0
895325,20171231,9,25076,4,1.0,1299.1700,1533.0206,1804.6494,2038.5


fct_cheque содержит данные о фактических продажах. Судя по значениям, **sale_grs** и **cost_grs** - цена закупки и продажи с НДС. Проверим, нет ли дублирующихся записей.

In [6]:
pd.read_sql('select * FROM fct_cheque order by date_id ', engine).duplicated().sum()

0

В таблице fct_cheque дубликатов нет.

In [7]:
pd.read_sql('select * FROM dim_date', engine)

,d,did,td,y,ty,q,tq,tqid,m,tm,tmid,tms,w,tw,twid,dw,tdw,day_type_id,day_type
0,2015-01-01,20150101,01.01.2015,2015,2015,20151,1 квартал,2015/1 Кв.,201501,Январь,2015/Январь,1,201501,00 Неделя,2015/00 Неделя,4,Четверг,1,Рабочий
1,2015-01-02,20150102,02.01.2015,2015,2015,20151,1 квартал,2015/1 Кв.,201501,Январь,2015/Январь,1,201501,00 Неделя,2015/00 Неделя,5,Пятница,1,Рабочий
2,2015-01-03,20150103,03.01.2015,2015,2015,20151,1 квартал,2015/1 Кв.,201501,Январь,2015/Январь,1,201501,00 Неделя,2015/00 Неделя,6,Суббота,2,Выходной
3,2015-01-04,20150104,04.01.2015,2015,2015,20151,1 квартал,2015/1 Кв.,201501,Январь,2015/Январь,1,201501,00 Неделя,2015/00 Неделя,7,Воскресенье,2,Выходной
4,2015-01-05,20150105,05.01.2015,2015,2015,20151,1 квартал,2015/1 Кв.,201501,Январь,2015/Январь,1,201502,01 Неделя,2015/01 Неделя,1,Понедельник,1,Рабочий
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2018-12-27,20181227,27.12.2018,2018,2018,20184,4 квартал,2018/4 Кв.,201812,Декабрь,2018/Декабрь,12,201852,51 Неделя,2018/51 Неделя,4,Четверг,1,Рабочий
1457,2018-12-28,20181228,28.12.2018,2018,2018,20184,4 квартал,2018/4 Кв.,201812,Декабрь,2018/Декабрь,12,201852,51 Неделя,2018/51 Неделя,5,Пятница,1,Рабочий
1458,2018-12-29,20181229,29.12.2018,2018,2018,20184,4 квартал,2018/4 Кв.,201812,Декабрь,2018/Декабрь,12,201852,51 Неделя,2018/51 Неделя,6,Суббота,2,Выходной
1459,2018-12-30,20181230,30.12.2018,2018,2018,20184,4 квартал,2018/4 Кв.,201812,Декабрь,2018/Декабрь,12,201852,51 Неделя,2018/51 Неделя,7,Воскресенье,2,Выходной


In [8]:
pd.read_sql('select * FROM dim_cash_register', engine)

,cash_register_id,cash_register_number
0,7,касса - 7
1,1,касса - 1
2,2,касса - 2
3,3,касса - 3
4,4,касса - 4
5,5,касса - 5
6,6,касса - 6
7,7,касса - 7
8,8,касса - 8
9,9,касса - 9


Здесь мы видим дублирующуюся запись "касса - 7"

In [9]:
pd.read_sql('select top 10 * FROM dim_stores', engine)

,store_id,store_name
0,1,Аптека - 1
1,2,Аптека - 2
2,3,Аптека - 3
3,4,Аптека - 4
4,5,Аптека - 5
5,6,Аптека - 6
6,7,Аптека - 7
7,8,Аптека - 8
8,9,Аптека - 9
9,10,Аптека - 10


In [10]:
pd.read_sql('select * FROM dim_stores', engine).duplicated().sum()

0

Дубликатов в dim_stores нет.

### Шаг 3: проверка данных

Проверим с помощью запроса без хранимых процедур, совпадут ли данные по продажам с данными заказчика

In [11]:
pd.read_sql("select sum(sale_grs) as [Продажи руб., с НДС], \
sum(quantity) as [Объем продаж, шт.]\
FROM fct_cheque where date_id between 20170601 and 20170630 \
and good_id in (select good_id from dim_goods where group_name = 'Биологически активные добавки')", engine)

,"Продажи руб., с НДС","Объем продаж, шт."
0,1782949.1,6761.1


Это совпало с данными заказчика 1 782 949.10 в рублях и 6761.10 в штуках. Значит, ошибка происходит при использовании хранимой процедуры.
##### Хранимая процедура

USE [TestTaskDB]
GO
/****** Object:  StoredProcedure [dbo].[sp_report_1]    Script Date: 08.07.2022 18:08:49 ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

ALTER PROCEDURE [dbo].[sp_report_1]
	@date_from date,
	@date_to date,
	@good_group_name nvarchar(MAX)
AS
BEGIN
		
	declare @date_from_int int
	declare @date_to_int int
	
	set @date_from_int = (select top 1 did from dbo.dim_date where d = @date_from )
	set @date_to_int = (select top 1 did from dbo.dim_date where d = @date_to )
	
	
	SELECT d.d as [Дата],
		s.store_name as [Аптека],
		g.group_name as [Группа товара] ,
		g.good_name as [Номенклатура],
		sum(f.quantity) as [Продажи шт.],
		sum(f.sale_net) as [Продажи руб., с НДС],
		sum(f.cost_net) as [Закупка руб., с НДС]		
	FROM [dbo].[fct_cheque] as f
	inner join dim_goods as g
		on g.good_id = f.good_id
	inner join [dim_stores] as s
		on s.store_id = f.store_id
	inner join dim_date as d
		on d.did = f.date_id
	inner join dbo.dim_cash_register as cr
		on cr.cash_register_id = f.cash_register_id
	where date_id between @date_from_int and @date_to_int
		and g.group_name = @good_group_name
	group by d.d,
		s.store_name,
		g.group_name,
		g.good_name

	
END


#### Запуск процедуры

In [12]:
df = pd.read_sql("exec sp_report_1 \
                 @date_from = '2017-06-01', \
                 @date_to = '2017-06-30', \
                 @good_group_name = N'Биологически активные добавки'", engine)
df

,Дата,Аптека,Группа товара,Номенклатура,Продажи шт.,"Продажи руб., с НДС","Закупка руб., с НДС"
0,2017-06-01,Аптека - 17,Биологически активные добавки,Гематогенка кокосовая 40г,2.0,46.7996,17.7796
1,2017-06-01,Аптека - 21,Биологически активные добавки,Аскорбиновая к-та с глюкозой бад таб. 1г №10,2.0,28.8876,6.1800
2,2017-06-01,Аптека - 25,Биологически активные добавки,Интернет-заказ,5.0,18944.6049,17579.4249
3,2017-06-01,Аптека - 25,Биологически активные добавки,Манжетка обыкновенная трава 50г,1.0,78.0448,38.6400
4,2017-06-01,Аптека - 29,Биологически активные добавки,Асвитол солнышко таб. жев. 25мг №10,3.0,112.6776,40.6800
...,...,...,...,...,...,...,...
3523,2017-06-30,Аптека - 3,Биологически активные добавки,Солгар легкодоступное железо джентл айрон капс...,1.0,1168.7772,723.4600
3524,2017-06-30,Аптека - 33,Биологически активные добавки,Саймы капс. 350мг №4,2.0,1979.4244,1225.4204
3525,2017-06-30,Аптека - 33,Биологически активные добавки,Худеем за неделю кофе жиросжигающий капуччино ...,1.0,452.7230,272.7700
3526,2017-06-30,Аптека - 37,Биологически активные добавки,Гематоген русский кедровый орех 40г,1.0,32.6708,12.9400


In [13]:
df['Продажи руб., с НДС'].sum()

1659486.9618000002

In [14]:
df['Продажи шт.'].sum()

6697.1

In [15]:
# Уникальных значений
df['Номенклатура'].nunique()

694

1 659 486.9618 в рублях и 6697.1 в штуках отличаются от ожидаемого результата: 1 782 949.10 в рублях и 6761.10 в штуках

Проверка хранимой процедуры дает тот же самый результат 1 659 486.9618 руб.

In [16]:
pd.read_sql("SELECT d.d as [Дата],\
s.store_name as [Аптека],\
g.group_name as [Группа товара],\
g.good_name as [Номенклатура],\
sum(f.quantity) as [Продажи шт.],\
sum(f.sale_net) as [Продажи руб., с НДС],\
sum(f.cost_net) as [Закупка руб., с НДС] \
FROM [dbo].[fct_cheque] as f \
inner join dim_goods as g \
on g.good_id = f.good_id \
inner join dim_stores as s \
on s.store_id = f.store_id \
inner join dim_date as d \
on d.did = f.date_id \
inner join dbo.dim_cash_register as cr \
on cr.cash_register_id = f.cash_register_id \
where date_id between 20170601 and 20170630 \
and g.group_name = N'Биологически активные добавки' \
group by d.d, \
s.store_name, \
g.group_name, \
g.good_name", engine)['Продажи руб., с НДС'].sum()

1659486.9618000002

### Шаг 4: поиск ошибки

Было выявлено:
1. В таблице fct_cheque значения с НДС - sale_grs и cost_grs. В хранимой процедуре используются sum(f.sale_net) и sum(f.cost_net), без НДС. Это необходимо исправить.
2. В таблицах имеются дубликаты. Необходимо исправить таблицы или SQL-запросы с учетом того, что имеются дубликаты.

In [17]:
pd.read_sql("SELECT d.d as [Дата]\
        , s.store_name as [Аптека]\
        , g.group_name as [Группа товара]\
        , g.good_name as [Номенклатура]\
        , SUM(f.quantity) as [Продажи шт.]\
        , SUM(f.sale_grs) as [Продажи руб., с НДС]\
        , SUM(f.cost_grs) as [Закупка руб., с НДС]\
    FROM [dbo].[fct_cheque] as f\
        LEFT JOIN dim_date as d\
        ON d.did=f.date_id\
        LEFT JOIN (SELECT DISTINCT * FROM dim_goods) as g\
        on f.good_id=g.good_id\
        LEFT JOIN dim_stores as s\
        on s.store_id=f.store_id\
    where date_id between 20170601 and 20170630 \
        and g.group_name='Биологически активные добавки'\
    GROUP BY d.d\
        , s.store_name \
        , g.group_name\
        , g.good_name", engine)[['Продажи руб., с НДС', 'Продажи шт.']].sum()

Продажи руб., с НДС    1782949.1
Продажи шт.               6761.1
dtype: float64

Это совпало с данными заказчика 1 782 949.10 в рублях и 6761.10 в штуках.

## Ответ на Задание 1

Причины расхождений:
1. В таблице fct_cheque значения с НДС - sale_grs и cost_grs. В хранимой процедуре используются sum(f.sale_net) и sum(f.cost_net), без НДС. Необходимо исправить cost_net на sale_grs и sale_net на sale_grs.
2. В таблицах имеются дубликаты. Необходимо удалить дубликаты или переписать хранимую процедуру.

**Исправленная хранимая процедура:**

USE [TestTaskDB]
GO
/****** Object:  StoredProcedure [dbo].[sp_report_1]    Script Date: 08.07.2022 18:08:49 ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

ALTER PROCEDURE [dbo].[sp_report_1]
	@date_from date,
	@date_to date,
	@good_group_name nvarchar(MAX)
AS
BEGIN
		
	declare @date_from_int int
	declare @date_to_int int
	
	set @date_from_int = (select top 1 did from dbo.dim_date where d = @date_from )
	set @date_to_int = (select top 1 did from dbo.dim_date where d = @date_to )


    SELECT d.d as [Дата]
        , s.store_name as [Аптека]
        , g.group_name as [Группа товара]
        , g.good_name as [Номенклатура]
        , SUM(f.quantity) as [Продажи шт.]
        , SUM(f.sale_grs) as [Продажи руб., с НДС]
        , SUM(f.cost_grs) as [Закупка руб., с НДС]
    FROM [dbo].[fct_cheque] as f
        LEFT JOIN dim_date as d
        ON d.did=f.date_id
        LEFT JOIN (SELECT DISTINCT * FROM dim_goods) as g
        on f.good_id=g.good_id
        LEFT JOIN dim_stores as s
        on s.store_id=f.store_id
    where date_id between @date_from and @date_to
        and g.group_name= @good_group_name
    GROUP BY d.d
        , s.store_name
        , g.group_name
        , g.good_name
    
END 

## Задание 2
В результат процедуры необходимо добавить показатели:

         -Средняя цена закупки руб., без НДС
         -Маржа руб. без НДС
         -Наценка % без НДС

In [18]:
pd.read_sql("SELECT d.d as [Дата]\
        , s.store_name as [Аптека]\
        , g.group_name as [Группа товара]\
        , g.good_name as [Номенклатура]\
        , SUM(f.quantity) as [Продажи шт.]\
        , SUM(f.sale_grs) as [Продажи руб., с НДС]\
        , SUM(f.cost_grs) as [Закупка руб., с НДС]\
        , AVG(f.cost_net) as [Средняя цена закупки руб., без НДС]\
        , (SUM(f.sale_net)-SUM(f.cost_net)) as [Маржа руб. без НДС]\
        , ((SUM(f.sale_net)-SUM(f.cost_net))/(NULLIF(SUM(f.cost_net),0))*100) as [Наценка % без НДС]\
    FROM [dbo].[fct_cheque] as f\
        LEFT JOIN dim_date as d\
        ON d.did=f.date_id\
        LEFT JOIN (SELECT DISTINCT * FROM dim_goods) as g\
        on f.good_id=g.good_id\
        LEFT JOIN dim_stores as s\
        on s.store_id=f.store_id\
    where date_id between 20170601 and 20170630 \
        and g.group_name='Биологически активные добавки'\
    GROUP BY d.d\
        , s.store_name \
        , g.group_name\
        , g.good_name", engine)

,Дата,Аптека,Группа товара,Номенклатура,Продажи шт.,"Продажи руб., с НДС","Закупка руб., с НДС","Средняя цена закупки руб., без НДС",Маржа руб. без НДС,Наценка % без НДС
0,2017-06-01,None,Биологически активные добавки,Янтарная кислота таб. 100мг №10,7.0,196.0,66.3278,56.2100,129.6722,230.69
1,2017-06-01,Аптека - 17,Биологически активные добавки,"Вечернее Биокор др. 0,18г №60",1.0,72.0,32.7686,27.7700,39.2314,141.27
2,2017-06-01,Аптека - 17,Биологически активные добавки,Интернет-заказ,1.0,647.0,600.7100,600.7100,46.2900,7.70
3,2017-06-01,Аптека - 21,Биологически активные добавки,Примадофилус детский пор. 50г №1,1.0,776.0,485.4048,411.3600,290.5952,70.64
4,2017-06-01,Аптека - 25,Биологически активные добавки,Масло косметическое облепиха 100мл,1.0,77.5,35.4236,30.0200,42.0764,140.16
...,...,...,...,...,...,...,...,...,...,...
3615,2017-06-30,Аптека - 43,Биологически активные добавки,Омела белая побеги 50г,2.0,140.0,62.4000,52.8814,77.6000,146.74
3616,2017-06-30,Аптека - 9,Биологически активные добавки,Гематоген русский кедровый орех 40г,3.0,111.0,49.6308,42.0600,61.3692,145.90
3617,2017-06-30,Аптека - 9,Биологически активные добавки,Гематоген русский новый лесной орех 40г,1.0,27.0,11.7764,9.9800,15.2236,152.54
3618,2017-06-30,Аптека - 9,Биологически активные добавки,"Лактогон таб. 0,55г №20",1.0,292.5,182.9118,155.0100,109.5882,70.69


## Ответ на Задание 2

Меняем хранимую процедуру exec sp_report_2:

USE [TestTaskDB]
GO
/****** Object:  StoredProcedure [dbo].[sp_report_2]    Script Date: 08.07.2022 18:08:49 ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

ALTER PROCEDURE [dbo].[sp_report_2]
	@date_from date,
	@date_to date,
	@good_group_name nvarchar(MAX)
AS
BEGIN
		
	declare @date_from_int int
	declare @date_to_int int
	
	set @date_from_int = (select top 1 did from dbo.dim_date where d = @date_from )
	set @date_to_int = (select top 1 did from dbo.dim_date where d = @date_to )


    SELECT d.d as [Дата]
        , s.store_name as [Аптека]
        , g.group_name as [Группа товара]
        , g.good_name as [Номенклатура]
        , SUM(f.quantity) as [Продажи шт.]
        , SUM(f.sale_grs) as [Продажи руб., с НДС]
        , SUM(f.cost_grs) as [Закупка руб., с НДС]
        , AVG(f.cost_net) as [Средняя цена закупки руб., без НДС]
        , (SUM(f.sale_net)-SUM(f.cost_net)) as [Маржа руб. без НДС]
        , ((SUM(f.sale_net)-SUM(f.cost_net))/(NULLIF(SUM(f.cost_net),0))*100) as [Наценка % без НДС]
    FROM [dbo].[fct_cheque] as f
        LEFT JOIN dim_date as d
        ON d.did=f.date_id
        LEFT JOIN (SELECT DISTINCT * FROM dim_goods) as g
        on f.good_id=g.good_id
        LEFT JOIN dim_stores as s
        on s.store_id=f.store_id
    where date_id between @date_from and @date_to
        and g.group_name= @good_group_name
    GROUP BY d.d
        , s.store_name 
        , g.group_name
        , g.good_name
    
END 

## Задание 3

Добавить возможность фильтрации по нескольким группам товаров одновременно. Пример входных данных: 'Биологически активные добавки,Косметические средства' Пример запуска процедуры после модернизации: exec sp_report_1 @date_from = '2017-06-01', @date_to = '2017-06-30', @good_group_name = 'Биологически активные добавки,Косметические средства'

## Ответ на Задание 3

Используем STRING_SPLIT с разделителем ','

In [19]:
pd.read_sql("SELECT d.d as [Дата]\
        , s.store_name as [Аптека]\
        , g.group_name as [Группа товара]\
        , g.good_name as [Номенклатура]\
        , SUM(f.quantity) as [Продажи шт.]\
        , SUM(f.sale_grs) as [Продажи руб., с НДС]\
        , SUM(f.cost_grs) as [Закупка руб., с НДС]\
        , AVG(f.cost_net) as [Средняя цена закупки руб., без НДС]\
        , (SUM(f.sale_net)-SUM(f.cost_net)) as [Маржа руб. без НДС]\
        , ((SUM(f.sale_net)-SUM(f.cost_net))/(NULLIF(SUM(f.cost_net),0))*100) as [Наценка % без НДС]\
    FROM [dbo].[fct_cheque] as f\
        LEFT JOIN dim_date as d\
        ON d.did=f.date_id\
        LEFT JOIN (SELECT DISTINCT * FROM dim_goods) as g\
        on f.good_id=g.good_id\
        LEFT JOIN dim_stores as s\
        on s.store_id=f.store_id\
    where date_id between 20170601 and 20170630 \
        and g.group_name in (select trim(value) from STRING_SPLIT('Биологически активные добавки,  Косметические средства',','))\
    GROUP BY d.d\
        , s.store_name \
        , g.group_name\
        , g.good_name", engine)

,Дата,Аптека,Группа товара,Номенклатура,Продажи шт.,"Продажи руб., с НДС","Закупка руб., с НДС","Средняя цена закупки руб., без НДС",Маржа руб. без НДС,Наценка % без НДС
0,2017-06-05,Аптека - 9,Косметические средства,З/щетка детская Орал би кидс,1.0,136.5,68.3967,57.9633,68.1033,117.49
1,2017-06-09,Аптека - 21,Биологически активные добавки,"Линекс д/детей пак. саше 1,5г №10",1.0,596.0,372.8446,315.9700,223.1554,70.62
2,2017-06-13,Аптека - 3,Биологически активные добавки,Вагилак капс. №15,1.0,920.0,609.5172,516.5400,310.4828,60.10
3,2017-06-10,Аптека - 43,Косметические средства,Зубочистки с нитью №30,1.0,165.0,87.4300,74.0932,77.5700,104.69
4,2017-06-29,Аптека - 43,Косметические средства,Гинокомфорт гель д/интим гигиены увлажняющий м...,1.0,555.0,367.8296,311.7200,187.1704,60.04
...,...,...,...,...,...,...,...,...,...,...
7634,2017-06-02,Аптека - 3,Биологически активные добавки,Феррогематоген пастилка жев. 50г (БАД),6.0,210.0,98.2800,41.6442,111.7200,134.13
7635,2017-06-19,Аптека - 3,Косметические средства,Санорин лорис спрей 30мл,1.0,470.0,319.5440,270.8000,150.4560,55.55
7636,2017-06-14,Аптека - 37,Биологически активные добавки,Феррогематоген пастилка жев. 50г (БАД),4.0,140.0,67.3544,57.0800,72.6456,127.26
7637,2017-06-16,Аптека - 25,Биологически активные добавки,Доппельгерц актив магний+калий таб. 1640мг №30,1.0,360.0,226.7370,192.1500,133.2630,69.35


**Создадим хранимую процедуру sp_report_2 и проверим вызов этой процедуры:**

USE [TestTaskDB]
GO
/****** Object:  StoredProcedure [dbo].[sp_report_2]    Script Date: 10.07.2022 0:25:53 ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author:		Alex Tiunov
-- Create date: 
-- Description:	
-- =============================================
ALTER PROCEDURE [dbo].[sp_report_2]
	@date_from date,
	@date_to date,
	@good_group_name nvarchar(MAX)
AS
BEGIN
	
		
	declare @date_from_int int
	declare @date_to_int int
	
	set @date_from_int = (select top 1 did from dbo.dim_date where d = @date_from )
	set @date_to_int = (select top 1 did from dbo.dim_date where d = @date_to )
	

		SELECT d.d as [Дата]
        , s.store_name as [Аптека]
        , g.group_name as [Группа товара]
        , g.good_name as [Номенклатура]
        , SUM(f.quantity) as [Продажи шт.]
        , SUM(f.sale_grs) as [Продажи руб., с НДС]
        , SUM(f.cost_grs) as [Закупка руб., с НДС]
        , AVG(f.cost_net) as [Средняя цена закупки руб., без НДС]
        , (SUM(f.sale_net)-SUM(f.cost_net)) as [Маржа руб. без НДС]
        , ((SUM(f.sale_net)-SUM(f.cost_net))/(NULLIF(SUM(f.cost_net),0))*100) as [Наценка % без НДС]
    FROM [dbo].[fct_cheque] as f
        LEFT JOIN dim_date as d
        ON d.did=f.date_id
        LEFT JOIN (SELECT DISTINCT * FROM dim_goods) as g
        on f.good_id=g.good_id
        LEFT JOIN dim_stores as s
        on s.store_id=f.store_id
    where date_id between @date_from_int and @date_to_int 
        and g.group_name in (select trim(value) from STRING_SPLIT(@good_group_name,','))
    GROUP BY d.d
        , s.store_name 
        , g.group_name
        , g.good_name
END


In [20]:
# Запускаем процедуру exec sp_report_2
pd.read_sql("exec sp_report_2 @date_from = '2017-06-01'\
, @date_to = '2017-06-30'\
, @good_group_name = 'Биологически активные добавки,Косметические средства'", engine)

,Дата,Аптека,Группа товара,Номенклатура,Продажи шт.,"Продажи руб., с НДС","Закупка руб., с НДС","Средняя цена закупки руб., без НДС",Маржа руб. без НДС,Наценка % без НДС
0,2017-06-01,None,Косметические средства,Жемене гиалуроновая к-та д/лица 10мл,1.0,285.0,178.2154,151.0300,106.7846,70.70
1,2017-06-01,Аптека - 17,Биологически активные добавки,"Вечернее Биокор др. 0,18г №60",1.0,72.0,32.7686,27.7700,39.2314,141.27
2,2017-06-01,Аптека - 17,Биологически активные добавки,Гематоген русский С 40г,1.0,23.0,7.8116,6.6200,15.1884,229.43
3,2017-06-01,Аптека - 17,Косметические средства,Масло косметическое персиковое 50мл,1.0,78.0,31.2600,26.4915,46.7400,176.43
4,2017-06-01,Аптека - 21,Биологически активные добавки,Витамин Е мирролла капс. 200мг №20,1.0,43.0,31.8010,26.9500,11.1990,41.55
...,...,...,...,...,...,...,...,...,...,...
7634,2017-06-30,Аптека - 9,Биологически активные добавки,Гематоген русский кедровый орех 40г,3.0,111.0,49.6308,42.0600,61.3692,145.90
7635,2017-06-30,Аптека - 9,Биологически активные добавки,Интернет-заказ,1.0,682.0,631.8300,631.8300,50.1700,7.94
7636,2017-06-30,Аптека - 9,Биологически активные добавки,Масло расторопши капс. №100,1.0,65.0,29.6298,25.1100,35.3702,140.86
7637,2017-06-30,Аптека - 9,Биологически активные добавки,Фитолакс таб. 500мг №20,1.0,227.5,142.4496,120.7200,85.0504,70.45


## Задание 4

Вывести долю продаж товара в %, относительно общего объема продаж в рублях в разрезе день/магазин/группа товаров, затем отсортировать выборку по убыванию показателя (нужна доля от продаж позиции в аптеке от продажи группы товара в этой же аптеке). Показатель добавить в текущую процедуру формирования данных для отчета. Расчет не должен выполняться с помощью вложенных запросов и CTE. Необходимо использовать функции Transact-SQL.

## Ответ на Задание 4
Составим запрос с OVER(PARTITION BY)

In [21]:
pd.read_sql("SELECT d.d as [Дата]\
        , s.store_name as [Аптека]\
        , g.group_name as [Группа товара]\
        , g.good_name as [Номенклатура]\
        , SUM(f.quantity) as [Продажи шт.]\
        , SUM(f.sale_grs) as [Продажи руб., с НДС]\
        , SUM(f.cost_grs) as [Закупка руб., с НДС]\
        , AVG(f.cost_net) as [Средняя цена закупки руб., без НДС]\
        , (SUM(f.sale_net)-SUM(f.cost_net)) as [Маржа руб. без НДС]\
        , ((SUM(f.sale_net)-SUM(f.cost_net))/(NULLIF(SUM(f.cost_net),0))*100) as [Наценка % без НДС]\
        , SUM(f.cost_grs)*100 / sum(f.cost_grs) OVER(PARTITION BY g.group_name order by d.d\
        , s.store_name) AS [доля продаж товара в %]\
        FROM [dbo].[fct_cheque] as f\
        LEFT JOIN dim_date as d\
        ON d.did=f.date_id\
        LEFT JOIN (SELECT DISTINCT * FROM dim_goods) as g\
        on f.good_id=g.good_id\
        LEFT JOIN dim_stores as s\
        on s.store_id=f.store_id\
    where date_id between 20170601 and 20170601 \
        and g.group_name in (select trim(value) from STRING_SPLIT('Биологически активные добавки,  Косметические средства',','))\
    GROUP BY d.d\
        , s.store_name \
        , g.group_name\
        , g.good_name\
        , f.cost_grs\
        order by d.d, s.store_name, g.group_name, [доля продаж товара в %] DESC", engine)

,Дата,Аптека,Группа товара,Номенклатура,Продажи шт.,"Продажи руб., с НДС","Закупка руб., с НДС","Средняя цена закупки руб., без НДС",Маржа руб. без НДС,Наценка % без НДС,доля продаж товара в %
0,2017-06-01,None,Биологически активные добавки,Интернет-заказ,2.0000,2029.1,1816.7100,1816.7100,212.3900,11.69,79.0055
1,2017-06-01,None,Биологически активные добавки,Омега-3-крилевый жир капс. №30,1.0000,642.0,401.3062,340.0900,240.6938,70.77,17.4521
2,2017-06-01,None,Биологически активные добавки,Янтарная кислота таб. 100мг №10,7.0000,196.0,66.3278,56.2100,129.6722,230.69,2.8844
3,2017-06-01,None,Биологически активные добавки,Гематоген русский кедровый орех 40г,1.0000,25.0,15.1276,12.8200,9.8724,77.00,0.6578
4,2017-06-01,None,Косметические средства,Жемене гиалуроновая к-та д/лица 10мл,1.0000,285.0,178.2154,151.0300,106.7846,70.70,100.0000
...,...,...,...,...,...,...,...,...,...,...,...
262,2017-06-01,Аптека - 9,Косметические средства,Пантенол эво крем универсал д/сухой поврежд ко...,1.0000,99.0,55.3656,46.9200,43.6344,92.99,0.1601
263,2017-06-01,Аптека - 9,Косметические средства,Помада гигиеническая Фруктовый поцелуй микс 9 ...,0.0556,77.0,54.9998,46.6100,22.0002,47.20,0.1591
264,2017-06-01,Аптека - 9,Косметические средства,Мирролла бальзам репейный д/укрепления волос 1...,1.0000,100.0,45.7800,38.7966,54.2200,139.75,0.1324
265,2017-06-01,Аптека - 9,Косметические средства,Бархатные ручки крем д/рук питательный масло ш...,1.0000,75.0,41.0758,34.8100,33.9242,97.45,0.1188


**Создадим хранимую процедуру:**

USE [TestTaskDB]
GO
/****** Object:  StoredProcedure [dbo].[sp_report_2]    Script Date: 10.07.2022 0:25:53 ******/
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
-- =============================================
-- Author:		Alex Tiunov
-- Create date: 
-- Description:	
-- =============================================
ALTER PROCEDURE [dbo].[sp_report_2]
	@date_from date,
	@date_to date,
	@good_group_name nvarchar(MAX)
AS
BEGIN
	
		
	declare @date_from_int int
	declare @date_to_int int
	
	set @date_from_int = (select top 1 did from dbo.dim_date where d = @date_from )
	set @date_to_int = (select top 1 did from dbo.dim_date where d = @date_to )
	

		SELECT d.d as [Дата]
        , s.store_name as [Аптека]
        , g.group_name as [Группа товара]
        , g.good_name as [Номенклатура]
        , SUM(f.quantity) as [Продажи шт.]
        , SUM(f.sale_grs) as [Продажи руб., с НДС]
        , SUM(f.cost_grs) as [Закупка руб., с НДС]
        , AVG(f.cost_net) as [Средняя цена закупки руб., без НДС]
        , (SUM(f.sale_net)-SUM(f.cost_net)) as [Маржа руб. без НДС]
        , ((SUM(f.sale_net)-SUM(f.cost_net))/(NULLIF(SUM(f.cost_net),0))*100) as [Наценка % без НДС]
        , SUM(f.cost_grs)*100 / sum(f.cost_grs) OVER(PARTITION BY g.group_name order by d.d
        , s.store_name) AS [доля продаж товара в %]
        FROM [dbo].[fct_cheque] as f
        LEFT JOIN dim_date as d
        ON d.did=f.date_id
        LEFT JOIN (SELECT DISTINCT * FROM dim_goods) as g
        on f.good_id=g.good_id
        LEFT JOIN dim_stores as s
        on s.store_id=f.store_id
    where date_id between @date_from_int and @date_to_int
        and g.group_name in (select trim(value) from STRING_SPLIT(@good_group_name,','))
    GROUP BY d.d
        , s.store_name 
        , g.group_name
        , g.good_name
        , f.cost_grs
        order by d.d, s.store_name, g.group_name, [доля продаж товара в %] DESC
END
